In [32]:
#Import all of the necessary libraries

import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import plotly.express as px
import numpy as np
import hvplot.pandas
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
#Load your .env file since we're using Alpaca's API keys
load_dotenv("alpaca_api_keys.env")

True

In [3]:
#Amount of shares are calculated based on the closing price of each stock on March 17, 2020
#Each stock has $12,500 worth

#BA closing price = $124.14
#DAL closing price = $31.74
#MGM closing price = $9.55
#WYNN closing price = $54.38

ba_shares = 101
dal_shares = 394
mgm_shares = 1309
wynn_shares = 230

In [4]:
#Create variables for each alpaca key to easily call it
alpaca_api_key = os.getenv("API_KEY")
alpaca_secret_key = os.getenv("API_SECRET_KEY")

In [5]:
#Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [6]:
#Set your portfolio study dates
start = pd.Timestamp("2020-03-17", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-04-13", tz="America/New_York").isoformat()

#Set your timeframe for the data
timeframe = "1D"

#Set your tickers
tickers = ["BA", "DAL", "MGM", "WYNN"]

In [7]:
#Set your dataframe and load the data
df_portfolio_pandemic = api.get_barset(
    tickers,
    timeframe,
    limit = 1000,
    start = start,
    end = end
).df

df_portfolio_pandemic.head()

BA                                      DAL  \
                             open    high     low   close    volume   open   
time                                                                         
2020-03-17 00:00:00-04:00  132.51  134.96  101.25  124.03  52088741  36.23   
2020-03-18 00:00:00-04:00   99.86  107.85   89.00  101.89  52497232  29.79   
2020-03-19 00:00:00-04:00   98.80  103.57   90.35   97.71  38196591  23.36   
2020-03-20 00:00:00-04:00   99.85  107.00   94.28   95.01  38848406  23.17   
2020-03-23 00:00:00-04:00   98.53  114.49   93.56  105.49  42338289  22.17   

                                                             MGM         \
                            high    low   close    volume   open   high   
time                                                                      
2020-03-17 00:00:00-04:00  37.19  30.60  31.740  31674028  10.92  11.01   
2020-03-18 00:00:00-04:00  29.79  19.10  23.485  56529508   9.00   9.03   
2020-03-19 00:00:00-04:00  24.15  20.00  21.500  42710188   7.17   9.24   
2020-03-20 00:00:00-04:00  26.85  21.26  21.350  52917970   8.89  11.16   
2020-03-23 00:00:00-04:00  22.83  20.60  22.230  41609238   9.73   9.82   

                                                    WYNN                  \
                              low close    volume   open   high      low   
time                                                                       
2020-03-17 00:00:00-04:00  9.0000  9.54  29102992  56.71  58.00  48.0100   
2020-03-18 00:00:00-04:00  5.9001  7.14  39987832  48.25  51.86  35.8400   
2020-03-19 00:00:00-04:00  6.4500  7.70  37067054  41.15  52.90  38.0100   
2020-03-20 00:00:00-04:00  8.6700  9.11  52910095  49.83  60.00  49.0101   
2020-03-23 00:00:00-04:00  8.9210  9.15  29228000  53.51  58.40  51.0000   

                                            
                           close    volume  
time                                        
2020-03-17 00:00:00-04:00  54.34   8532019  
2020-03-18 00:00:00-04:00  43.02  19940992  
2020-03-19 00:00:00-04:00  46.57  10201487  
2020-03-20 00:00:00-04:00  51.97  11225096  
2020-03-23 00:00:00-04:00  57.56   8383887

In [8]:
BA_portfolio = df_portfolio_pandemic['BA'][['close']]

In [9]:
DAL_portfolio = df_portfolio_pandemic['DAL'][['close']]

In [10]:
MGM_portfolio = df_portfolio_pandemic['MGM'][['close']]

In [11]:
WYNN_portfolio = df_portfolio_pandemic['WYNN'][['close']]

In [12]:
pandemic_portfolio = pd.concat([BA_portfolio, DAL_portfolio, MGM_portfolio, WYNN_portfolio], axis = 'columns', join = 'inner')

pandemic_portfolio.columns = ['BA', 'DAL', 'MGM', 'WYNN']

pandemic_portfolio.head(10)

BA     DAL     MGM   WYNN
time                                                    
2020-03-17 00:00:00-04:00  124.03  31.740   9.540  54.34
2020-03-18 00:00:00-04:00  101.89  23.485   7.140  43.02
2020-03-19 00:00:00-04:00   97.71  21.500   7.700  46.57
2020-03-20 00:00:00-04:00   95.01  21.350   9.110  51.97
2020-03-23 00:00:00-04:00  105.49  22.230   9.150  57.56
2020-03-24 00:00:00-04:00  127.50  26.890  12.185  66.54
2020-03-25 00:00:00-04:00  158.68  31.140  12.945  75.06
2020-03-26 00:00:00-04:00  180.40  31.700  13.410  71.40
2020-03-27 00:00:00-04:00  161.99  29.550  12.180  63.31
2020-03-30 00:00:00-04:00  152.27  28.670  11.530  61.06

In [13]:
pandemic_portfolio.to_csv('Pandemic_Stocks_Raw_Data.csv')

In [14]:
pandemic_portfolio = pd.concat([BA_portfolio, DAL_portfolio, MGM_portfolio, WYNN_portfolio], axis = 'columns', join = 'inner')

pandemic_portfolio.columns = ['BA', 'DAL', 'MGM', 'WYNN']

pandemic_portfolio.head(10)

BA     DAL     MGM   WYNN
time                                                    
2020-03-17 00:00:00-04:00  124.03  31.740   9.540  54.34
2020-03-18 00:00:00-04:00  101.89  23.485   7.140  43.02
2020-03-19 00:00:00-04:00   97.71  21.500   7.700  46.57
2020-03-20 00:00:00-04:00   95.01  21.350   9.110  51.97
2020-03-23 00:00:00-04:00  105.49  22.230   9.150  57.56
2020-03-24 00:00:00-04:00  127.50  26.890  12.185  66.54
2020-03-25 00:00:00-04:00  158.68  31.140  12.945  75.06
2020-03-26 00:00:00-04:00  180.40  31.700  13.410  71.40
2020-03-27 00:00:00-04:00  161.99  29.550  12.180  63.31
2020-03-30 00:00:00-04:00  152.27  28.670  11.530  61.06

In [15]:
pandemic_portfolio.to_csv('Pandemic_Stocks_Raw_Data.csv')

In [17]:
final_portfolio = pd.read_csv("../../data/cleandata/Final_Portfolio.csv", index_col = "Date", parse_dates = True, infer_datetime_format = True)
final_portfolio.head(10)

SP500       Nasdaq      BA     DAL     MGM   WYNN    Tesla  \
Date                                                                          
2020-03-17  252.800003  7334.779785  124.03  31.740   9.540  54.34  430.000   
2020-03-18  240.000000  6989.839844  101.89  23.485   7.140  43.02  361.600   
2020-03-19  240.509995  7150.580078   97.71  21.500   7.700  46.57  427.295   
2020-03-20  228.800003  6879.520020   95.01  21.350   9.110  51.97  427.530   
2020-03-23  222.949997  6860.669922  105.49  22.230   9.150  57.56  434.290   
2020-03-24  243.149994  7417.859863  127.50  26.890  12.185  66.54  504.300   
2020-03-25  246.789993  7384.299805  158.68  31.140  12.945  75.06  539.175   
2020-03-26  261.200012  7797.540039  180.40  31.700  13.410  71.40  528.050   
2020-03-27  253.419998  7502.379883  161.99  29.550  12.180  63.31  514.560   
2020-03-30  261.649994  7774.149902  152.27  28.670  11.530  61.06  502.100   

            Gamestop     AAPL      AMZN      BRK.A    MSFT          BTC  \
Date                                                                      
2020-03-17    4.2300  252.850  1807.380  280619.12  146.57  5225.629395   
2020-03-18    3.7800  249.000  1832.390  258927.00  142.13  5238.438477   
2020-03-19    4.1900  244.780  1880.500  261690.00  142.67  6191.192871   
2020-03-20    3.7550  228.740  1846.090  254600.00  136.77  6198.778320   
2020-03-23    3.8100  223.430  1902.540  242000.00  135.56  6416.314941   
2020-03-24    4.1950  246.880  1940.000  267090.00  148.34  6734.803711   
2020-03-25    4.2000  245.465  1885.780  269409.75  146.85  6681.062988   
2020-03-26    4.4050  258.480  1955.225  276095.00  156.11  6716.440430   
2020-03-27    4.2101  247.620  1899.920  269350.00  149.61  6469.798340   
2020-03-30    3.6500  254.790  1964.780  275052.50  160.30  6429.841797   

                   ETH  
Date                    
2020-03-17  113.942749  
2020-03-18  114.842270  
2020-03-19  136.593857  
2020-03-20  132.737167  
2020-03-23  134.911606  
2020-03-24  138.761444  
2020-03-25  136.195892  
2020-03-26  138.361557  
2020-03-27  133.937943  
2020-03-30  132.904541

In [18]:
final_portfolio.dtypes

SP500       float64
Nasdaq      float64
BA          float64
DAL         float64
MGM         float64
WYNN        float64
Tesla       float64
Gamestop    float64
AAPL        float64
AMZN        float64
BRK.A       float64
MSFT        float64
BTC         float64
ETH         float64
dtype: object

In [19]:
daily_returns = final_portfolio.pct_change()
daily_returns

SP500    Nasdaq        BA       DAL       MGM      WYNN  \
Date                                                                     
2020-03-17       NaN       NaN       NaN       NaN       NaN       NaN   
2020-03-18 -0.050633 -0.047028 -0.178505 -0.260082 -0.251572 -0.208318   
2020-03-19  0.002125  0.022996 -0.041025 -0.084522  0.078431  0.082520   
2020-03-20 -0.048688 -0.037907 -0.027633 -0.006977  0.183117  0.115954   
2020-03-23 -0.025568 -0.002740  0.110304  0.041218  0.004391  0.107562   
2020-03-24  0.090603  0.081215  0.208645  0.209627  0.331694  0.156011   
2020-03-25  0.014970 -0.004524  0.244549  0.158051  0.062372  0.128043   
2020-03-26  0.058390  0.055962  0.136879  0.017983  0.035921 -0.048761   
2020-03-27 -0.029786 -0.037853 -0.102051 -0.067823 -0.091723 -0.113305   
2020-03-30  0.032476  0.036225 -0.060004 -0.029780 -0.053366 -0.035539   
2020-03-31 -0.014905 -0.009525 -0.020687 -0.004883  0.023851 -0.014248   
2020-04-01 -0.045005 -0.044093 -0.124061 -0.164739 -0.003812 -0.102841   
2020-04-02  0.023075  0.017217 -0.057342 -0.047839 -0.021548 -0.025924   
2020-04-03 -0.014454 -0.015256  0.012263 -0.009255 -0.080962 -0.079279   
2020-04-06  0.067166  0.073261  0.190870 -0.007562  0.219858  0.171381   
2020-04-07  0.001019 -0.003283 -0.046621 -0.002913  0.056589  0.074211   
2020-04-08  0.033568  0.025819  0.037453  0.044729  0.100514  0.132425   
2020-04-09  0.015217  0.007747  0.034058  0.049699 -0.030167  0.023765   
2020-04-13 -0.009130  0.004764 -0.028786 -0.046936 -0.044509 -0.019108   
2020-04-14  0.029493  0.039466 -0.043747  0.055699  0.082014  0.073593   
2020-04-15 -0.021248 -0.014392  0.034754 -0.008348 -0.041556 -0.034140   
2020-04-16  0.004824  0.016583 -0.080644 -0.064507 -0.041970  0.007375   
2020-04-20  0.008921  0.003325  0.070496  0.038208 -0.020275  0.012123   
2020-04-21 -0.030363 -0.034752 -0.050634 -0.025381 -0.026608 -0.038047   
2020-04-22  0.022195  0.028094 -0.009244 -0.022135  0.011769 -0.005250   
2020-04-23 -0.000072 -0.000074  0.018956 -0.002219  0.033396  0.084724   
2020-04-24  0.013939  0.016454 -0.062568 -0.003114  0.013072  0.007101   
2020-04-27  0.014418  0.011077 -0.003798 -0.011156  0.090323  0.080820   
2020-04-28 -0.004598 -0.014024  0.018676  0.098150  0.039448 -0.007490   
2020-04-29  0.026178  0.035663  0.061645  0.122457  0.103732  0.083496   
...              ...       ...       ...       ...       ...       ...   
2021-03-02 -0.007803 -0.016929 -0.005436  0.003537  0.006268  0.008347   
2021-03-03 -0.013246 -0.027026  0.024474  0.002488  0.008046  0.020394   
2021-03-04 -0.012375 -0.021102 -0.016845 -0.028542 -0.039907 -0.019249   
2021-03-05  0.018397  0.015458 -0.006587 -0.016181 -0.004291  0.002594   
2021-03-08 -0.004979 -0.024070  0.003091  0.036139  0.019391  0.002663   
2021-03-09  0.014278  0.036851  0.029967 -0.005639 -0.009775  0.002394   
2021-03-10  0.006225 -0.000382  0.063757  0.003991  0.004269  0.006269   
2021-03-11  0.010139  0.025239  0.027173  0.010879  0.040914  0.022249   
2021-03-12  0.001347 -0.005882  0.067979  0.031457 -0.005360 -0.005949   
2021-03-15  0.005964  0.010499 -0.013117  0.022873  0.050808  0.020289   
2021-03-16 -0.001261  0.000881 -0.039120 -0.031189 -0.025641 -0.016595   
2021-03-17  0.003410  0.003981  0.032601  0.021462  0.033083  0.017603   
2021-03-18 -0.014550 -0.030242 -0.027930 -0.025173 -0.041485 -0.030379   
2021-03-19 -0.005109  0.007553 -0.001327 -0.003662  0.013161 -0.001253   
2021-03-22  0.007985  0.012281 -0.018138 -0.021427 -0.025231 -0.019782   
2021-03-23 -0.007871 -0.011201 -0.039414 -0.048175 -0.043311 -0.057380   
2021-03-24 -0.005083 -0.020095 -0.008994 -0.000657 -0.011787 -0.000160   
2021-03-25  0.005626  0.001218  0.034545  0.030476  0.029005  0.023650   
2021-03-26  0.016115  0.012410 -0.010066  0.014043 -0.007771 -0.006478   
2021-03-29 -0.000505 -0.006019  0.023563 -0.007134 -0.013939 -0.029775   
2021-03-30 -0.002653 -0.001092  0.006024  0.033601  0.029079  0.015628   
2021-03-31  0.00405

In [20]:
correlation = daily_returns.corr()
correlation

SP500    Nasdaq        BA       DAL       MGM      WYNN  \
SP500     1.000000  0.909747  0.612336  0.545959  0.590228  0.495306   
Nasdaq    0.909747  1.000000  0.452562  0.336834  0.463659  0.366758   
BA        0.612336  0.452562  1.000000  0.780275  0.634832  0.613152   
DAL       0.545959  0.336834  0.780275  1.000000  0.728344  0.692758   
MGM       0.590228  0.463659  0.634832  0.728344  1.000000  0.819116   
WYNN      0.495306  0.366758  0.613152  0.692758  0.819116  1.000000   
Tesla     0.355504  0.438829  0.238801  0.174732  0.251777  0.239368   
Gamestop  0.040006  0.029228  0.115109  0.090588  0.026749  0.028045   
AAPL      0.360271  0.377402  0.171820  0.122642  0.198406  0.147971   
AMZN      0.499329  0.717580  0.084059 -0.034851  0.101003  0.056002   
BRK.A     0.786950  0.558860  0.630350  0.669960  0.639930  0.552093   
MSFT      0.781285  0.885797  0.280835  0.159567  0.283246  0.193567   
BTC       0.213486  0.266551  0.135331  0.071424  0.143374  0.110065   
ETH       0.214360  0.249957  0.132757  0.039906  0.095323  0.079669   

             Tesla  Gamestop      AAPL      AMZN     BRK.A      MSFT  \
SP500     0.355504  0.040006  0.360271  0.499329  0.786950  0.781285   
Nasdaq    0.438829  0.029228  0.377402  0.717580  0.558860  0.885797   
BA        0.238801  0.115109  0.171820  0.084059  0.630350  0.280835   
DAL       0.174732  0.090588  0.122642 -0.034851  0.669960  0.159567   
MGM       0.251777  0.026749  0.198406  0.101003  0.639930  0.283246   
WYNN      0.239368  0.028045  0.147971  0.056002  0.552093  0.193567   
Tesla     1.000000  0.003019  0.772182  0.321711  0.184196  0.395845   
Gamestop  0.003019  1.000000 -0.045236 -0.025477  0.049356  0.023258   
AAPL      0.772182 -0.045236  1.000000  0.288503  0.191005  0.413054   
AMZN      0.321711 -0.025477  0.288503  1.000000  0.121083  0.706467   
BRK.A     0.184196  0.049356  0.191005  0.121083  1.000000  0.419618   
MSFT      0.395845  0.023258  0.413054  0.706467  0.419618  1.000000   
BTC       0.181403  0.033716  0.079482  0.157317  0.132048  0.172583   
ETH       0.106673  0.067342  0.000337  0.150171  0.123210  0.192686   

               BTC       ETH  
SP500     0.213486  0.214360  
Nasdaq    0.266551  0.249957  
BA        0.135331  0.132757  
DAL       0.071424  0.039906  
MGM       0.143374  0.095323  
WYNN      0.110065  0.079669  
Tesla     0.181403  0.106673  
Gamestop  0.033716  0.067342  
AAPL      0.079482  0.000337  
AMZN      0.157317  0.150171  
BRK.A     0.132048  0.123210  
MSFT      0.172583  0.192686  
BTC       1.000000  0.727166  
ETH       0.727166  1.000000